## Inportaciones

In [140]:
import pyodbc
import pandas as pd
import sqlite3
import datetime

Se debe cambiar a la dirección correspondiente

In [127]:
db_path = r"C:\Users\dilon\OneDrive\Escritorio\Proyectos\Proyecto-Capstone\POS\BackEnd\db.sqlite3"

## Conexión

In [3]:
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost;"
    "DATABASE=POSREF;"
    "Trusted_Connection=yes;"
)

# Conexión
try:
    conn = pyodbc.connect(conn_str)
    print("Conexión exitosa.")
except Exception as e:
    print("Error de conexión:", e)

Conexión exitosa.


Ma_familia 1 son platos y menus 

Ma_familia 2 son papas

Ma_familia 3 son bebidas

Ma_familia 4 son descuentos (Los remplace por extras)

## Extract de Productos

In [78]:
df = pd.read_sql("SELECT * FROM LVENTA1", conn)

query = """
SELECT 
    a.Ma_descripcion AS Articulo_descripcion, 
    CASE a.Ma_familia
        WHEN 1 THEN 'Pollos y Menús'
        WHEN 2 THEN 'Papas'
        WHEN 3 THEN 'Bebidas'
        WHEN 4 THEN 'Descuentos'
        ELSE 'Sin categoría'
    END AS categoria,
    a.Ma_proporcion AS proporcion,
    p.Ma_preconiva AS precio_con_iva
FROM ARTICULOS a
LEFT JOIN LPRECIOS2 p
    ON  a.Ma_Emp = p.Ma_Emp
    AND a.Ma_ref = p.Ma_ref
    AND p.Ma_lista = 1;   -- aquí filtras lista 1
"""

df = pd.read_sql(query, conn)
print("Filas leídas:", len(df))
df.head(20)

Filas leídas: 47


C:\Users\dilon\AppData\Local\Temp\ipykernel_9552\725722888.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM LVENTA1", conn)
C:\Users\dilon\AppData\Local\Temp\ipykernel_9552\725722888.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,Articulo_descripcion,categoria,proporcion,precio_con_iva
0,POLLO ENTERO,Pollos y Menús,1.00,14990.0
1,BEBIDA LATA,Bebidas,NaN,1300.0
2,BEBIDA 1 1/2 LITROS,Bebidas,NaN,2300.0
3,BEBIDA 2 LITROS,Bebidas,NaN,2000.0
4,BEBIDA 2 1/2 LITROS,Bebidas,NaN,2500.0
5,BEBIDA 3 LITROS,Bebidas,NaN,3300.0
6,COLACION PARA SERVIR,Pollos y Menús,0.25,3200.0
7,COLACION COSTILLAR,Bebidas,0.00,5000.0
8,COLACION PARA LLEVAR,Pollos y Menús,0.25,5990.0
9,MENU 1 (1/4 POLLO + PAPAS),Pollos y Menús,0.25,NaN


## Transformaciones

In [79]:
df["proporcion"] = df["proporcion"].fillna(0)

In [80]:
df["Articulo_descripcion"] = df["Articulo_descripcion"].str.capitalize()

In [82]:
df = df.dropna(subset=["precio_con_iva"])

In [85]:
df.head(10)

,Articulo_descripcion,categoria,proporcion,precio_con_iva
0,Pollo entero,Pollos y Menús,1.00,14990.0
1,Bebida lata,Bebidas,0.00,1300.0
2,Bebida 1 1/2 litros,Bebidas,0.00,2300.0
3,Bebida 2 litros,Bebidas,0.00,2000.0
4,Bebida 2 1/2 litros,Bebidas,0.00,2500.0
5,Bebida 3 litros,Bebidas,0.00,3300.0
6,Colacion para servir,Pollos y Menús,0.25,3200.0
7,Colacion costillar,Bebidas,0.00,5000.0
8,Colacion para llevar,Pollos y Menús,0.25,5990.0
11,1/2 pollo,Pollos y Menús,0.50,7990.0


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43 entries, 0 to 46
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Id_articulo           43 non-null     int64  
 1   Articulo_descripcion  43 non-null     object 
 2   categoria             43 non-null     object 
 3   proporcion            43 non-null     float64
 4   precio_sin_iva        43 non-null     float64
 5   precio_con_iva        43 non-null     float64
dtypes: float64(3), int64(1), object(2)
memory usage: 2.4+ KB


In [87]:
df["Articulo_descripcion"] = df["Articulo_descripcion"].astype(str)
df["categoria"] = df["categoria"].astype(str)

## Filtrado de datos importantes

In [113]:
df_filtrado = df[
    ~df["Articulo_descripcion"].str.contains(
        "colacion|colación|menu|menú|despacho|bandeja|vaso|promo|1/2 pollo\\+papas medianas\\+bebida 1,5",
        case=False,
        na=False,
        regex=True
    )
]

In [139]:
df_filtrado.head(30)

,Articulo_descripcion,categoria,proporcion,precio_con_iva
6,Colacion para servir,Pollos y Menús,0.25,3200.0
7,Colacion costillar,Bebidas,0.00,5000.0
8,Colacion para llevar,Pollos y Menús,0.25,5990.0
13,Menu 4 (1/2 pollo + papas medianas,Pollos y Menús,0.50,11490.0
14,Menu 5 (1 pollo entero + papa),Pollos y Menús,1.00,18990.0
15,Menu 6 (pollo entero+ papas +bebida1.5),Pollos y Menús,1.00,20990.0
16,Menu 7 (pollo entero+papas+bebida 1.5),Pollos y Menús,1.00,23990.0
35,"1/2 pollo+papas medianas+bebida 1,5",Bebidas,0.00,13490.0
36,Bandeja pequeña,Bebidas,0.00,300.0
37,Bandeja grande,Bebidas,0.00,500.0


## Creación de tabla item_id

In [116]:
df_filtrado["item_id"] = 3 

df_filtrado.loc[
    df_filtrado["Articulo_descripcion"].str.contains("pollo", case=False, na=False),
    "item_id"
] = 1

df_filtrado.loc[
    df_filtrado["Articulo_descripcion"].str.contains("bebida", case=False, na=False),
    "item_id"
] = 2

C:\Users\dilon\AppData\Local\Temp\ipykernel_9552\1343297376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["item_id"] = 3


## Reajuste de la tabla categoria

In [ ]:
df_filtrado.loc[
    df_filtrado["categoria"].str.contains("Pollo", case=False, na=False),
    "categoria"
] = 1

df_filtrado.loc[
    df_filtrado["categoria"].str.contains("Bebidas", case=False, na=False),
    "categoria"
] = 2
df_filtrado.loc[
    df_filtrado["categoria"].str.contains("Papas", case=False, na=False),
    "categoria"
] = 3

df_filtrado.loc[
    df_filtrado["categoria"].str.contains("Descuentos", case=False, na=False),
    "categoria"
] = 2

In [122]:
df_filtrado["categoria"] = df_filtrado["categoria"].astype(int)

C:\Users\dilon\AppData\Local\Temp\ipykernel_9552\2964957041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["categoria"] = df_filtrado["categoria"].astype(int)


## Creación de tabla is_active

In [124]:
df_filtrado["is_active"] = 1 

C:\Users\dilon\AppData\Local\Temp\ipykernel_9552\2485050183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["is_active"] = 1


## Inicialización de la base de datos sqlite 3 insersión de datos

In [128]:
conn_sqlite = sqlite3.connect(db_path)
cursor = conn_sqlite.cursor()

In [131]:
rows = []
for _, row in df_filtrado.iterrows():
    rows.append((
        row["precio_con_iva"],          # precio
        int(row["is_active"]),          # is_active (0/1)
        int(row["item_id"]),            # item_id
        float(row["proporcion"]),       # eq_pollo
        row["categoria"],               # categoria_id
        row["Articulo_descripcion"],    # descripcion
    ))

In [132]:
insert_sql = """
INSERT INTO inventario_producto
(precio, is_active, item_id, eq_pollo, categoria_id, descripcion)
VALUES (?, ?, ?, ?, ?, ?)
"""

In [133]:
try:
    cursor.executemany(insert_sql, rows)
    conn_sqlite.commit()
    conn_sqlite.close()
    print("Carga a inventario_producto completada.")
except Exception as e:
    print(f"Error durante la carga a inventario_producto: {e}")

Carga a inventario_producto completada.


## Extracción y actualización de datos para tabla de promociones

In [138]:
df_filtrado = df[
    df["Articulo_descripcion"].str.contains(
        "colacion|colación|menu|menú|bandeja|promo|1/2 pollo\\+papas medianas\\+bebida 1,5",
        case=False,
        na=False,
        regex=True
    )
]

In [143]:
df_filtrado["fecha_inicio"] = datetime.datetime.now().date()

C:\Users\dilon\AppData\Local\Temp\ipykernel_9552\493676880.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["fecha_inicio"] = datetime.datetime.now().date()


In [144]:
df_filtrado.head(10)

,Articulo_descripcion,categoria,proporcion,precio_con_iva,fecha_inicio
6,Colacion para servir,Pollos y Menús,0.25,3200.0,2025-11-19
7,Colacion costillar,Bebidas,0.00,5000.0,2025-11-19
8,Colacion para llevar,Pollos y Menús,0.25,5990.0,2025-11-19
13,Menu 4 (1/2 pollo + papas medianas,Pollos y Menús,0.50,11490.0,2025-11-19
14,Menu 5 (1 pollo entero + papa),Pollos y Menús,1.00,18990.0,2025-11-19
15,Menu 6 (pollo entero+ papas +bebida1.5),Pollos y Menús,1.00,20990.0,2025-11-19
16,Menu 7 (pollo entero+papas+bebida 1.5),Pollos y Menús,1.00,23990.0,2025-11-19
35,"1/2 pollo+papas medianas+bebida 1,5",Bebidas,0.00,13490.0,2025-11-19
36,Bandeja pequeña,Bebidas,0.00,300.0,2025-11-19
37,Bandeja grande,Bebidas,0.00,500.0,2025-11-19


In [145]:
# Eliminar las columnas 'categoria' y 'proporcion' del dataframe filtrado
df_filtrado = df_filtrado.drop(columns=['categoria', 'proporcion'])
df_filtrado.head()

,Articulo_descripcion,precio_con_iva,fecha_inicio
6,Colacion para servir,3200.0,2025-11-19
7,Colacion costillar,5000.0,2025-11-19
8,Colacion para llevar,5990.0,2025-11-19
13,Menu 4 (1/2 pollo + papas medianas,11490.0,2025-11-19
14,Menu 5 (1 pollo entero + papa),18990.0,2025-11-19


In [146]:
rows = []
for _, row in df_filtrado.iterrows():
    rows.append((
        row["precio_con_iva"],          # precio
        row["fecha_inicio"],            # fecha_inicio
        row["Articulo_descripcion"],    # descripcion
    ))

Apertura de sqlite e insersión de datos

In [147]:
insert_sql = """
INSERT INTO inventario_promocion
(precio, fecha_inicio, descripcion)
VALUES (?, ?, ?)
"""

In [149]:
conn_sqlite = sqlite3.connect(db_path)
cursor = conn_sqlite.cursor()

In [150]:
try:
    cursor.executemany(insert_sql, rows)
    conn_sqlite.commit()
    conn_sqlite.close()
    print("Carga a inventario_promocion completada.")
except Exception as e:
    print(f"Error durante la carga a inventario_promocion: {e}")

Carga a inventario_promocion completada.
